# Using LLama Factory finetune on SageMaker 
# 1. Single GPU QLORA- 本地notebook实例训练

## 安装依赖包

In [18]:
%pip install -Uq sagemaker boto3 datasets

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pip/__main__.py", line 8, in <module>
    if sys.path[0] in ("", os.getcwd()):
FileNotFoundError: [Errno 2] No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install torch==2.2.0

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pip/__main__.py", line 8, in <module>
    if sys.path[0] in ("", os.getcwd()):
FileNotFoundError: [Errno 2] No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
import glob
import boto3
import pprint
from tqdm import tqdm
import sagemaker
from sagemaker.collection import Collection
from sagemaker.utils import name_from_base

In [21]:
sagemaker_session =  sagemaker.session.Session() #sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
sm_client = boto3.client('sagemaker', region_name=region)

## 准备数据集

### 数据集1. 从huggingface上下载ruozhiba数据集
- 改数据集有近5k条数据，本次实验我们可以只用前1k条做训练

In [22]:
from datasets import load_dataset
from random import randrange
dataset_name = "hfl/ruozhiba_gpt4"
# Load dataset from the hub
train_dataset = load_dataset(dataset_name, split="train",revision='41d2c61beb86c8d4c61916cc656c39d018c40ce5')

print(f"Training size: {len(train_dataset)}")
print("\nTraining sample:\n")
print(train_dataset[randrange(len(train_dataset))])

Training size: 4898

Training sample:

{'input': '', 'instruction': '怎么才能把牙齿弄干净 到底要吃多少管牙膏才行', 'output': '把牙齿弄干净的方法很多，关键在于正确的口腔护理，而不是吃多少管牙膏。正确的刷牙方式包括用合适的牙刷和牙膏，采用正确的刷牙手法，每次刷牙至少两分钟，每天刷两次。此外，使用牙线、漱口水等工具帮助清洁牙齿缝隙也很重要。定期去牙医那里进行专业清洁和检查也是保持牙齿健康的重要措施。至于‘吃’牙膏，这是一个误解，牙膏的作用是在刷牙过程中帮助去除牙菌斑和牙垢，吃下牙膏不但没用，还有可能对身体有害，因为很多牙膏含有氟化物等化学成分，不适合内服。因此，保养牙齿重点在于正确的刷牙习惯和护理，而不是牙膏的量。'}


### 数据集2. 身份数据集
```json
[{'instruction': 'hi',
  'input': '',
  'output': 'Hello! I am {{name}}, an AI assistant developed by {{author}}. How can I assist you today?'},
 {'instruction': 'hello',
  'input': '',
  'output': 'Hello! I am {{name}}, an AI assistant developed by {{author}}. How can I assist you today?'},
 {'instruction': 'Who are you?',
  'input': '',
  'output': 'I am {{name}}, an AI assistant developed by {{author}}. How can I assist you today?'}]
```
把其中的name和author替换成您自己想替换的值，这样微调完成之后，问模型“你是谁，谁创造的你？”这类的身份问题，模型就会按这个新的值来回答

In [23]:
def format_identity(origin_obj,name,author):
    ret = []
    for ele in origin_obj:
        ele['output'] = ele['output'].replace("{{name}}",name).replace("{{author}}",author)
        ret.append(ele)
    return ret

- 替换成您自己的设定

In [24]:
NAME = 'AKABOT'
AUTHOR = 'AWS'

In [25]:
!pwd
%cd ~/SageMaker/Easy_Fintune_LLM_using_SageMaker_with_LLama_Factory

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
pwd: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
pwd: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/home/ec2-user/SageMaker/Easy_Fintune_LLM_using_SageMaker_with_LLama_Factory


In [26]:
!pwd

/home/ec2-user/SageMaker/Easy_Fintune_LLM_using_SageMaker_with_LLama_Factory


In [27]:
import json
file_name = './LLaMA-Factory/data/identity.json'
with open(file_name) as f:
    identity = json.load(f)
identity_2 = format_identity(identity,name=NAME,author=AUTHOR)
identity_2[:2]

[{'instruction': 'hi',
  'input': '',
  'output': 'Hello! I am AKABOT, an AI assistant developed by AWS. How can I assist you today?'},
 {'instruction': 'hello',
  'input': '',
  'output': 'Hello! I am AKABOT, an AI assistant developed by AWS. How can I assist you today?'}]

In [28]:
os.makedirs('./train',exist_ok=True)
with open('./train/identity_2.json','w') as f:
    json.dump(identity_2,f)

### 把数据copy至S3

In [30]:
s3_data_uri = f"s3://{default_bucket}/dataset-for-training"
training_input_path = f'{s3_data_uri}/train'

In [31]:
print(training_input_path)

s3://sagemaker-us-east-1-342367142984/dataset-for-training/train


In [32]:
# save train_dataset to s3
train_dataset.to_json('./train/ruozhiba.json')
sagemaker.s3.S3Uploader.upload(local_path="./train/ruozhiba.json", desired_s3_uri=training_input_path, sagemaker_session=sagemaker_session)
sagemaker.s3.S3Uploader.upload(local_path="./train/identity_2.json", desired_s3_uri=training_input_path, sagemaker_session=sagemaker_session)

print(f"saving training dataset to: {training_input_path}")


Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

saving training dataset to: s3://sagemaker-us-east-1-342367142984/dataset-for-training/train


## 准备LLaMA-Factory 的 dataset info

In [33]:
import json

In [34]:
file_name = './LLaMA-Factory/data/dataset_info.json'
with open(file_name) as f:
    datainfo = json.load(f)

In [35]:
datainfo['identity']={'file_name': 'identity_2.json'}

In [36]:
datainfo['ruozhiba']={
    'file_name':'ruozhiba.json',
    "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output",
  }      
}

In [37]:
with open('./LLaMA-Factory/data/dataset_info.json','w') as f:
    json.dump(fp=f,obj=datainfo)

## 准备LLaMA-Factory 的 训练配置yaml文件
###  从LLaMA-Factory/examples/train_qlora/目录中复制出llama3_lora_sft_awq.yaml，并修改

In [38]:
#load template
import yaml
file_name = './LLaMA-Factory/examples/train_qlora/llama3_lora_sft_awq.yaml'
with open(file_name) as f:
    doc = yaml.safe_load(f)
doc

{'model_name_or_path': 'TechxGenus/Meta-Llama-3-8B-Instruct-AWQ',
 'stage': 'sft',
 'do_train': True,
 'finetuning_type': 'lora',
 'lora_target': 'all',
 'dataset': 'identity,alpaca_en_demo',
 'template': 'llama3',
 'cutoff_len': 1024,
 'max_samples': 1000,
 'overwrite_cache': True,
 'preprocessing_num_workers': 16,
 'output_dir': 'saves/llama3-8b/lora/sft',
 'logging_steps': 10,
 'save_steps': 500,
 'plot_loss': True,
 'overwrite_output_dir': True,
 'per_device_train_batch_size': 1,
 'gradient_accumulation_steps': 8,
 'learning_rate': 0.0001,
 'num_train_epochs': 3.0,
 'lr_scheduler_type': 'cosine',
 'warmup_ratio': 0.1,
 'fp16': True,
 'ddp_timeout': 180000000,
 'val_size': 0.1,
 'per_device_eval_batch_size': 1,
 'eval_strategy': 'steps',
 'eval_steps': 500}

In [39]:
#设置模型的保存目录在本notebook实例本地
save_dir = '/home/ec2-user/SageMaker/Easy_Fintune_LLM_using_SageMaker_with_LLama_Factory/finetuned_model'
# doc['output_dir'] = save_dir

# 如果是用SageMaker则使用以下模型文件路径
doc['output_dir'] ='/tmp/finetuned_model'
doc['per_device_train_batch_size'] =1
doc['gradient_accumulation_steps'] =8
# doc['lora_target'] = 'all'
doc['cutoff_len'] = 2048
doc['num_train_epochs'] = 5.0
doc['warmup_steps'] = 10

#实验时间，只选取前200条数据做训练
doc['max_samples'] = 200 
#数据集
doc['dataset'] = 'identity,ruozhiba'

### 保存为训练配置文件

In [40]:
sg_config = 'sg_config_qlora.yaml'
with open(f'./LLaMA-Factory/{sg_config}', 'w') as f:
    yaml.safe_dump(doc, f)
doc

{'model_name_or_path': 'TechxGenus/Meta-Llama-3-8B-Instruct-AWQ',
 'stage': 'sft',
 'do_train': True,
 'finetuning_type': 'lora',
 'lora_target': 'all',
 'dataset': 'identity,ruozhiba',
 'template': 'llama3',
 'cutoff_len': 2048,
 'max_samples': 200,
 'overwrite_cache': True,
 'preprocessing_num_workers': 16,
 'output_dir': '/tmp/finetuned_model',
 'logging_steps': 10,
 'save_steps': 500,
 'plot_loss': True,
 'overwrite_output_dir': True,
 'per_device_train_batch_size': 1,
 'gradient_accumulation_steps': 8,
 'learning_rate': 0.0001,
 'num_train_epochs': 5.0,
 'lr_scheduler_type': 'cosine',
 'warmup_ratio': 0.1,
 'fp16': True,
 'ddp_timeout': 180000000,
 'val_size': 0.1,
 'per_device_eval_batch_size': 1,
 'eval_strategy': 'steps',
 'eval_steps': 500,
 'warmup_steps': 10}

## 本地GPU测试提交 Training job

### 由于我们的实验环境限制，无法提交Training Job，所以在本次实验是在notebook实例中进行训练
### 如果您在自己的AWS环境中，且有SageMaker Training Job 所需GPU实例的quota，则可以用如下代码提交，instance_type改成'ml.g5.2xlarge' 

```python
from sagemaker.estimator import Estimator
from sagemaker.pytorch import PyTorch
from datetime import datetime

instance_count = 1
instance_type = 'local_gpu' 
max_time = 3600*24

# Get the current time
current_time = datetime.now()

# wandb.sagemaker_auth(path="./")
# Format the current time as a string
formatted_time = current_time.strftime("%Y%m%d%H%M%S")
print(formatted_time)

base_job_name = 'llama3-8b-qlora-finetune'
environment = {
    'NODE_NUMBER':str(instance_count),
    "s3_data_paths":f"{training_input_path}",
    "sg_config":sg_config,
    'OUTPUT_MODEL_S3_PATH': f's3://{default_bucket}/llama3-8b-qlora/', # destination
}

estimator = PyTorch(entry_point='entry_single_lora.py',
                            source_dir='./LLaMA-Factory/',
                            role=role,
                            base_job_name=base_job_name,
                            environment=environment,
                            framework_version='2.2.0',
                            py_version='py310',
                            script_mode=True,
                            instance_count=instance_count,
                            instance_type=instance_type,
                            enable_remote_debug=True,
                            # keep_alive_period_in_seconds=600,
                            max_run=max_time)

estimator.fit()

```


In [41]:
from sagemaker.estimator import Estimator
from sagemaker.pytorch import PyTorch
from datetime import datetime

instance_count = 1

#使用本地机器，也可以指定为 ml.g5.2xlarge等其他实例
instance_type = 'local_gpu' 
max_time = 3600*24

# Get the current time
current_time = datetime.now()

# wandb.sagemaker_auth(path="./")
# Format the current time as a string
formatted_time = current_time.strftime("%Y%m%d%H%M%S")
print(formatted_time)

base_job_name = 'llama3-8b-qlora-finetune'
environment = {
    'NODE_NUMBER':str(instance_count),
    "s3_data_paths":f"{training_input_path}",
    "sg_config":sg_config,
    'OUTPUT_MODEL_S3_PATH': f's3://{default_bucket}/llama3-8b-qlora/', # destination
}

estimator = PyTorch(entry_point='entry_single_lora.py',
                            source_dir='./LLaMA-Factory/',
                            role=role,
                            base_job_name=base_job_name,
                            environment=environment,
                            framework_version='2.2.0',
                            py_version='py310',
                            script_mode=True,
                            instance_count=instance_count,
                            instance_type=instance_type,
                            enable_remote_debug=True,
                            # keep_alive_period_in_seconds=600,
                            max_run=max_time)

20240710155757


- 开始训练

In [ ]:
estimator.fit()

INFO:sagemaker.local.image:image pulled: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.2.0-gpu-py310
INFO:sagemaker.local.image:docker command: docker-compose -f /tmp/tmp1_2cnjfj/docker-compose.yaml up --build --abort-on-container-exit


 Container 5k8gfq1mzx-algo-1-cuufd  Creating
 Container 5k8gfq1mzx-algo-1-cuufd  Created
Attaching to 5k8gfq1mzx-algo-1-cuufd
5k8gfq1mzx-algo-1-cuufd  | 2024-07-10 16:02:12,396 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
5k8gfq1mzx-algo-1-cuufd  | 2024-07-10 16:02:12,430 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
5k8gfq1mzx-algo-1-cuufd  | 2024-07-10 16:02:12,439 sagemaker-training-toolkit INFO     instance_groups entry not present in resource_config
5k8gfq1mzx-algo-1-cuufd  | 2024-07-10 16:02:12,442 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
5k8gfq1mzx-algo-1-cuufd  | 2024-07-10 16:02:12,445 sagemaker_pytorch_container.training INFO     Invoking user training script.
5k8gfq1mzx-algo-1-cuufd  | 2024-07-10 16:02:13,600 botocore.credentials INFO     Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
5k8gfq1mzx-algo-1-cuufd  | 2024-07

INFO:root:creating /tmp/tmp1_2cnjfj/artifacts/output/data
INFO:root:copying /tmp/tmp1_2cnjfj/algo-1-cuufd/output/success -> /tmp/tmp1_2cnjfj/artifacts/output
INFO:sagemaker.local.image:===== Job Complete =====


5k8gfq1mzx-algo-1-cuufd exited with code 0
Aborting on container exit...
 Container 5k8gfq1mzx-algo-1-cuufd  Stopping
 Container 5k8gfq1mzx-algo-1-cuufd  Stopped


## 至此部，本章节结束
- 模型已经在本地训练完成，并上传至s3 位置在 : s3://{default_bucket}/llama3-8b-qlora/

### 以下是可选步骤，直接在本地使用LLaMA-Factory cli进行训练
### 本地运行LLaMA-Factory cli

In [20]:
#切换工作目录到LLaMA-Factory
os.chdir('LLaMA-Factory')

In [28]:
#安装LLaMA-Factory
os.system("pip install --no-deps -e .")

Obtaining file:///home/ec2-user/SageMaker/llm_finetune/LLaMA-Factory
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for llamafactory (pyproject.toml): started
  Building editable for llamafactory (pyproject.toml): finished with status 'done'
  Created wheel for llamafactory: filename=llamafactory-0.7.2.dev0-0.editable-py3-none-any.whl size=18819 sha256=3cbaa1b62e626d21787b17927d017a1567e30e626d3d4d138a226c374e0b5ee4
  Sto

0

In [29]:
os.system("pip install -r requirements.txt")

  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-0.30.1-py3-none-any.whl.metadata (18 kB)
  Using cached peft-0.11.1-py3-none-any.whl.metadata (13 kB)
  Using cached gradio-4.33.0-py3-none-any.whl.metadata (15 kB)
  Using cached einops-0.8.0-py3-none-any.whl.metadata (12 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached uvicorn-0.30.1-py3-none-any.whl.metadata (6.3 kB)
  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Using cached sse_starlette-2.1.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached fire-0.6.0-py2.py3-none-any.whl
  Using cached deepspeed-0.14.2-py3-none-any.whl
  Using cached autoawq-0.2.5-cp310-cp310-manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached metrics-0.3.3.tar.gz (18 kB)
  Preparing metadata (setup.py): sta

0

In [30]:
#下载数据集
os.system("chmod +x ./s5cmd")
os.system("./s5cmd sync {0} {1}".format(training_input_path+'/*', 'data/'))

cp s3://sagemaker-us-east-1-434444145045/dataset-for-training/train/identity_2.json data/identity_2.json
cp s3://sagemaker-us-east-1-434444145045/dataset-for-training/train/ruozhiba.json data/ruozhiba.json


0

### 启动训练
本次训练过程大概15分钟左右

In [8]:
DEVICES=0
os.system(f"CUDA_VISIBLE_DEVICES={DEVICES} llamafactory-cli train {sg_config}")

[2024-06-06 15:15:02,102] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[INFO|tokenization_utils_base.py:2108] 2024-06-06 15:15:04,360 >> loading file tokenizer.json from cache at /home/ec2-user/.cache/huggingface/hub/models--TechxGenus--Meta-Llama-3-8B-Instruct-AWQ/snapshots/129d90727841a07bcdb3173ed4165d1353b44386/tokenizer.json
[INFO|tokenization_utils_base.py:2108] 2024-06-06 15:15:04,360 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2108] 2024-06-06 15:15:04,360 >> loading file special_tokens_map.json from cache at /home/ec2-user/.cache/huggingface/hub/models--TechxGenus--Meta-Llama-3-8B-Instruct-AWQ/snapshots/129d90727841a07bcdb3173ed4165d1353b44386/special_tokens_map.json
[INFO|tokenization_utils_base.py:2108] 2024-06-06 15:15:04,360 >> loading file tokeni

 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
06/06/2024 15:15:04 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16


[WARNING|logging.py:314] 2024-06-06 15:15:04,678 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


06/06/2024 15:15:04 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>
06/06/2024 15:15:04 - INFO - llamafactory.data.template - Add pad token: <|eot_id|>
06/06/2024 15:15:04 - INFO - llamafactory.data.loader - Loading dataset identity_2.json...


...

input_ids:
[128000, 128006, 9125, 128007, 271, 2675, 527, 264, 11190, 18328, 13, 128009, 128006, 882, 128007, 271, 6151, 128009, 128006, 78191, 128007, 271, 9906, 0, 358, 1097, 11188, 24406, 11, 459, 15592, 18328, 8040, 555, 480, 12501, 12501, 46, 13, 2650, 649, 358, 7945, 499, 3432, 30, 128009]
inputs:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

hi<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hello! I am RiverBot, an AI assistant developed by GOGOGO. How can I assist you today?<|eot_id|>
label_ids:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 9906, 0, 358, 1097, 11188, 24406, 11, 459, 15592, 18328, 8040, 555, 480, 12501, 12501, 46, 13, 2650, 649, 358, 7945, 499, 3432, 30, 128009]
labels:
Hello! I am RiverBot, an AI assistant developed by GOGOGO. How can I assist you today?<|eot_id|>
06/06/202

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.42it/s]
[INFO|modeling_utils.py:4280] 2024-06-06 15:15:11,325 >> All model checkpoint weights were used when initializing LlamaForCausalLM.

[INFO|modeling_utils.py:4288] 2024-06-06 15:15:11,325 >> All the weights of LlamaForCausalLM were initialized from the model checkpoint at TechxGenus/Meta-Llama-3-8B-Instruct-AWQ.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
[INFO|configuration_utils.py:917] 2024-06-06 15:15:11,347 >> loading configuration file generation_config.json from cache at /home/ec2-user/.cache/huggingface/hub/models--TechxGenus--Meta-Llama-3-8B-Instruct-AWQ/snapshots/129d90727841a07bcdb3173ed4165d1353b44386/generation_config.json
[INFO|configuration_utils.py:962] 2024-06-06 15:15:11,347 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    12

06/06/2024 15:15:11 - INFO - llamafactory.model.utils.checkpointing - Gradient checkpointing enabled.
06/06/2024 15:15:11 - INFO - llamafactory.model.utils.attention - Using torch SDPA for faster training and inference.
06/06/2024 15:15:11 - INFO - llamafactory.model.adapter - Upcasting trainable params to float32.
06/06/2024 15:15:11 - INFO - llamafactory.model.adapter - Fine-tuning method: LoRA
06/06/2024 15:15:11 - INFO - llamafactory.model.loader - trainable params: 3407872 || all params: 1054347264 || trainable%: 0.3232


[INFO|trainer.py:2078] 2024-06-06 15:15:11,667 >> ***** Running training *****
[INFO|trainer.py:2079] 2024-06-06 15:15:11,667 >>   Num examples = 261
[INFO|trainer.py:2080] 2024-06-06 15:15:11,667 >>   Num Epochs = 5
[INFO|trainer.py:2081] 2024-06-06 15:15:11,668 >>   Instantaneous batch size per device = 1
[INFO|trainer.py:2084] 2024-06-06 15:15:11,668 >>   Total train batch size (w. parallel, distributed & accumulation) = 8
[INFO|trainer.py:2085] 2024-06-06 15:15:11,668 >>   Gradient Accumulation steps = 8
[INFO|trainer.py:2086] 2024-06-06 15:15:11,668 >>   Total optimization steps = 160
[INFO|trainer.py:2087] 2024-06-06 15:15:11,669 >>   Number of trainable parameters = 3,407,872
  1%|          | 1/160 [00:06<17:22,  6.56s/it]Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/bin/llamafactory-cli", line 8, in <module>
    sys.exit(main())
  File "/home/ec2-user/SageMaker/llm_finetune/LLaMA-Factory/src/llamafactory/cli.py", line 93, in main
    run_

2

### 上传Lora模型文件至S3保存

In [18]:
os.system("./s5cmd sync {0} {1}".format(save_dir, f's3://{default_bucket}/llama3-8b-qlora/'))

0

In [19]:
print(f"Lora model file saved s3://{default_bucket}/llama3-8b-qlora/")

Lora model file saved s3://sagemaker-us-east-1-434444145045/llama3-8b-qlora/
